In [1]:
import sys
import importlib
import os
import matplotlib.pyplot as plt
import pandas as pd
# make the module available from the src directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.dataLoaders.PatientDataLoader import PatientDataProcessor
importlib.reload(sys.modules['src.dataLoaders.PatientDataLoader'])

<module 'src.dataLoaders.PatientDataLoader' from '/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py'>

In [3]:
meta_folder = 'OAIData21/'
data_path = '../datasets/meta_data/' + meta_folder


processor = PatientDataProcessor(base_path=data_path)
# processor.create_meta_data_for_visit(visit_no=visit)
processor.load_all_visits()
all_clinical_data = processor.get_all_clinical_data()
kmri_data = processor. load_all_kMRI_data()



Loading kMRI data for visit V00
object
Length of the dataframe: 329
Length of the dataframe: 329
Loading kMRI data for visit V01
object
Length of the dataframe: 227
Length of the dataframe: 227
Loading kMRI data for visit V02
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_Eckstein02.txt skipping.
Length of the dataframe: 0
Loading kMRI data for visit V03
object
Length of the dataframe: 189
Length of the dataframe: 189
Loading kMRI data for visit V04
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_Eckstein04.txt skipping.
Length of the dataframe: 0
Loading kMRI data for visit V05
object
Length of the dataframe: 20
Length of the dataframe: 20
Loading kMRI data for visit V06
object
Length of the dataframe: 45
Length of the dataframe: 45
Loading kMRI data for visit V07
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_Eckstein07.txt skipping.
Length of the dataframe: 0
Loading kMRI data for visit V08
No file found at:  ../datasets/meta_data/OAIDa

/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:172: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.enroll_df = df_enroll[['P02SEX']].replace(
/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:311: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method

Missing columns for visit V01: {'V01XRKL'}
Missing columns for visit V02: {'V02XRKL'}
Missing columns for visit V03: {'V03XRKL'}


/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:172: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.enroll_df = df_enroll[['P02SEX']].replace(
/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:311: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method

Missing columns for visit V04: {'V04XRKL'}
Missing columns for visit V05: {'V05XRKL'}
Missing columns for visit V06: {'V06XRKL'}
Missing columns for visit V07: {'V07XRKL'}


/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:172: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.enroll_df = df_enroll[['P02SEX']].replace(
/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:311: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method

Missing columns for visit V08: {'V08XRKL'}
Missing columns for visit V09: {'V09XRKL'}


/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:172: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.enroll_df = df_enroll[['P02SEX']].replace(
/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:311: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method

Missing columns for visit V10: {'V10XRKL'}
Missing columns for visit V11: {'V11XRKL'}
Loading kMRI data for visit V00
object
Length of the dataframe: 329
Loading kMRI data for visit V01
object
Length of the dataframe: 227
Loading kMRI data for visit V02
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_Eckstein02.txt skipping.
Loading kMRI data for visit V03
object
Length of the dataframe: 189
Loading kMRI data for visit V04
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_Eckstein04.txt skipping.
Loading kMRI data for visit V05
object
Length of the dataframe: 20
Loading kMRI data for visit V06
object
Length of the dataframe: 45
Loading kMRI data for visit V07
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_Eckstein07.txt skipping.
Loading kMRI data for visit V08
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_Eckstein08.txt skipping.
Loading kMRI data for visit V09
No file found at:  ../datasets/meta_data/OAIData21//kMRI_QCart_E

/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:172: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.enroll_df = df_enroll[['P02SEX']].replace(
/Users/mraabo/computer_science/year_3/bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py:311: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method

In [8]:
all_clinical_data

,V00AGE,V00WOMKPL,V00WOMKPR,V01AGE,V01WOMKPL,V01WOMKPR,V02AGE,V02WOMKPL,V02WOMKPR,V03AGE,...,V08WOMKPR,V09AGE,V09WOMKPL,V09WOMKPR,V10AGE,V10WOMKPL,V10WOMKPR,V11AGE,V11WOMKPL,V11WOMKPR
ID,,,,,,,,,,,,,,,,,,,,,
9000099,59,0.0,1.0,60.0,0.0,0.0,NaN,NaN,NaN,61.0,...,4.0,65,0.0,0.0,66.0,4.0,3.0,67.0,4.0,2.0
9000296,69,0.0,0.0,70.0,0.0,0.0,NaN,NaN,NaN,71.0,...,0.0,76,0.0,0.0,77.0,0.0,0.0,78.0,0.0,0.0
9000622,71,0.0,3.0,72.0,0.0,2.0,NaN,NaN,NaN,63.3,...,2.2,,,,68.9,2.1,2.2,69.4,2.1,2.3
9000798,56,8.0,0.0,58.0,8.0,0.0,58.0,7.0,0.0,59.0,...,0.0,63,3.0,0.0,64.0,5.0,0.0,65.0,3.0,0.0
9001104,72,0.0,4.0,73.0,0.0,1.0,NaN,NaN,NaN,74.0,...,2.2,,,,80.0,0.0,1.0,69.4,2.1,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999365,56,4.0,5.0,57.0,3.0,5.0,NaN,NaN,NaN,58.0,...,3.0,63,7.0,1.0,64.0,4.0,2.0,65.0,5.0,6.0
9999510,50,3.0,0.0,51.0,1.0,0.0,NaN,NaN,NaN,52.0,...,1.0,57,1.0,1.0,58.0,3.0,0.0,59.0,4.0,0.0
9999862,61,0.0,0.0,62.0,0.0,0.0,NaN,NaN,NaN,63.0,...,0.0,68,0.0,0.0,69.0,0.0,0.0,70.0,0.0,0.0


In [7]:
kmri_data

,V00BLFPDL,V00ALTPDL,V00IBMFPDL,V00BLFPDR,V00ALTPDR,V00IBMFPDR,V01BLFPDL,V01ALTPDL,V01IBMFPDL,V01BLFPDR,...,V05IBMFPDL,V05BLFPDR,V05ALTPDR,V05IBMFPDR,V06BLFPDL,V06ALTPDL,V06IBMFPDL,V06BLFPDR,V06ALTPDR,V06IBMFPDR
ID,,,,,,,,,,,,,,,,,,,,,
9000798,9.9144,0.0,25.7054,0.000000,0.000,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9001897,0.0000,0.0,0.0000,0.000000,0.000,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9002430,5.0250,0.0,28.0461,6.488000,0.000,10.225000,5.6417,0.0,34.45150,7.334000,...,49.4082,7.151,0.0,15.7820,10.8169,0.0,45.0273,7.836,0.0,19.694
9011918,0.0000,0.0,0.0000,0.000000,0.000,0.000000,0.0000,0.0,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9021102,0.0000,0.0,0.0000,0.000000,0.000,0.000000,0.0000,0.0,0.00000,0.000000,...,NaN,NaN,NaN,NaN,0.0000,0.0,0.0000,0.000,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986207,0.0000,0.0,0.0000,0.000000,0.000,0.000000,0.0000,0.0,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9993650,0.0000,0.0,6.6430,0.000000,0.000,0.000000,0.0000,0.0,11.78180,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9993833,0.0000,0.0,9.6465,0.000000,0.000,49.939175,0.0000,0.0,7.61775,0.000000,...,7.6269,0.000,0.0,40.8897,NaN,NaN,NaN,NaN,NaN,NaN
